In [1]:
import time

from tqdm import tqdm
import numpy as np

In [2]:
## prepare blocks.
blocks = []
#blocks.append(np.zeros((4, 4)).astype(int))

blocks_ = np.zeros((4, 4)).astype(int)
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks_[3, 2] = 1
blocks_[3, 3] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0]) 
blocks_ = np.zeros((4, 3)).astype(int)
blocks_[1, 1] = 1
blocks_[2, 0] = 1
blocks_[2, 1] = 1
blocks_[2, 2] = 1
blocks_[3, 1] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0])
blocks_ = np.zeros((4, 3)).astype(int)
blocks_[1, 2] = 1
blocks_[2, 2] = 1
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks_[3, 2] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0]) 
blocks_ = np.zeros((4, 1)).astype(int)
blocks_[0, 0] = 1
blocks_[1, 0] = 1
blocks_[2, 0] = 1
blocks_[3, 0] = 1
blocks.append(blocks_)

#blocks_ = np.copy(blocks[0])
blocks_ = np.zeros((4, 2)).astype(int)
blocks_[2, 0] = 1
blocks_[2, 1] = 1
blocks_[3, 0] = 1
blocks_[3, 1] = 1
blocks.append(blocks_)

blocks

[array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 1, 1]]),
 array([[0, 0, 0],
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]]),
 array([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]]),
 array([[1],
        [1],
        [1],
        [1]]),
 array([[0, 0],
        [0, 0],
        [1, 1],
        [1, 1]])]

In [8]:
def get_n_col(pattern):
    return np.shape(blocks[pattern])[1]

    
def set_block(field, pattern, n_row, n_col, k):
    '''
    add the pattern*k to the field.
    (x, y) is the left-bottom position of the block.
    '''
    field_ = np.copy(field)
    block = blocks[pattern]
    for r in range(n_row, n_row+4):
        for c in range(n_col, n_col+get_n_col(pattern)):
            field_[r][c] += block[r-n_row][c-n_col] * k
    return field_


def move_block(field, pattern, n_row, n_col, jet):
    if jet == '<':
        if movable(field, pattern, n_row, n_col-1, n_row, n_col):
            field = set_block(field, pattern, n_row, n_col, -1)
            n_col -= 1
            field = set_block(field, pattern, n_row, n_col, 1)
    elif jet == '>':
        if movable(field, pattern, n_row, n_col+1, n_row, n_col):
            field = set_block(field, pattern, n_row, n_col, -1)
            n_col += 1
            field = set_block(field, pattern, n_row, n_col, 1)
            
    return field, n_col


def movable(field, pattern, n_row_next, n_col_next, n_row, n_col):
    if all([n_row_next >= 0, n_row_next < np.shape(field)[0]-3, n_col_next >=0, n_col_next < 7-get_n_col(pattern)+1]):
        field_ = set_block(field, pattern, n_row, n_col, -1)
        field_ = set_block(field_, pattern, n_row_next, n_col_next, 1)
        if np.max(field_) > 1:
            is_movable = False
        else:
            is_movable = True
    else:
        is_movable = False
    return is_movable


def drop_block(field, pattern, n_row, n_col):
    field = set_block(field, pattern, n_row, n_col, -1)
    n_row += 1
    field = set_block(field, pattern, n_row, n_col, 1)
    return field, n_row


def rest_block(field, pattern, jets, n_jet):
    n_row = 0
    n_col = 2    
    field = set_block(field, pattern, n_row, n_col, 1)

    while(1):
        n_jet = n_jet % len(jets)
        field, n_col = move_block(field, pattern, n_row, n_col, jets[n_jet])
        n_jet += 1

        if movable(field, pattern, n_row+1, n_col, n_row, n_col):
            field, n_row = drop_block(field, pattern, n_row, n_col)
        else:
            break
    return field, n_jet


def get_tower_height(field):
    # get height of the tower.
    n_row_max, n_col_max = np.shape(field)
    nonzero = []
    # check the height of each column.
    for i in range(n_col_max):
        try: 
            x = list(field[:, i]!=0).index(1)
        except ValueError:
            x = n_row_max
        nonzero.append(x)

    block_top    = np.min(nonzero)
    block_bottom = np.max(nonzero)
    
    return block_top, block_bottom, nonzero


def update_field(field):
    n_col_max = 7
    block_top, block_bottom, _ = get_tower_height(field)
    n_rows_to_add = 7 - block_top
    filler = np.zeros((n_rows_to_add, n_col_max)).astype(int)
    field_ = np.vstack((filler, field[:block_bottom, :]))

    return field_, np.shape(field)[0] - block_bottom


def calc_height(jets, n_blocks_max, take_records=False):
    n_col_max = 7
    n_row_max = 7
    pattern = 0
    n_jet = 0
    block_removed = 0
    field = np.zeros((n_row_max, n_col_max)).astype(int) 
    records = np.zeros((1, n_col_max+2)).astype(int)
    
    #for n_block in tqdm(range(0, n_blocks_max)):
    for n_block in range(0, n_blocks_max):
        #print("\r"+str(n_block), end="")

        if n_block > 0:
            field, block_removed_ = update_field(field)
            block_removed += block_removed_
            n_row_max = np.shape(field)[1]
            pattern = n_block % 5

        field, n_jet = rest_block(field, pattern, jets, n_jet)

        # for part2
        if take_records:
            block_top, _, heights = get_tower_height(field)
            heights -= block_top
            record = np.append(heights, [n_jet, pattern])
            records = np.vstack((records, record))
            records_unique = np.unique(records, axis=0)
            if len(records) != len(records_unique):
                break

    block_top, _, _ = get_tower_height(field)

    return (np.shape(field)[0] - block_top) + block_removed, records

In [9]:
is_test  = True
if is_test:
    jets = list('>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>')
else:
    with open('input.txt') as f:
        jets = f.read().strip()

## part 1. 
n_blocks_max = 2022
tower_height, _ = calc_height(jets, n_blocks_max, take_records=False)
print(f"tower height: {tower_height}")

tower height: 3068


In [10]:
## part 2.
# first check if the length of the jets is a prime number.
with open('input.txt') as f:
    jets = f.read().strip()
n = len(jets)

def prime_factorize(n):
    a = []
    while n % 2 == 0:
        a.append(2)
        n //= 2
    f = 3
    while f * f <= n:
        if n % f == 0:
            a.append(f)
            n //= f
        else:
            f += 2
    if n != 1:
        a.append(n)
    return a

prime_factorize(n) # 10091 is a prime number.

[10091]

In [11]:
is_test  = True
if is_test:
    jets = list('>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>')
else:
    with open('input.txt') as f:
        jets = f.read().strip()


n_blocks_max = 1000000000000
_, records = calc_height(jets, n_blocks_max, take_records=True)

# records is the list of height(array). 
#records = records[1:, :]
row = records[-1, :]
ids = np.where((records == row).all(axis=1))[0]
row_id1 = ids[0]
row_id2 = ids[1]
row_repeat = row_id2 - row_id1
n_repeat = (n_blocks_max-row_id1) // row_repeat
remain   = (n_blocks_max-row_id1) % row_repeat
print(f"{n_blocks_max} = {row_id1} + {n_repeat} x {row_repeat} + {remain}")

# test
#27 + 28571428570 * 35 + 23
# real
#104 + 589970501 x 1695 + 701
height1, _ = calc_height(jets, row_id1)
height2, _ = calc_height(jets, row_id2)
height_repeat = height2 - height1
height_remain, _ = calc_height(jets, row_id1 + remain) - height1

height1 + height_repeat * n_repeat + height_remain
# 1514285714288

1000000000000 = 28 + 28571428570 x 35 + 22


/tmp/ipykernel_661/1286758457.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  height_remain, _ = calc_height(jets, row_id1 + remain) - height1


1514285714288